## **1. Постановка задачи**
**Цель**: Построить систему предсказания оттока клиентов телеком-компании.

**Данные**: 
- Исторические данные о клиентах (CSV, 50+ млн строк)
- Логи взаимодействий (JSON, 1+ TB в S3)
- Внешние данные о регионах (Parquet)

**Требования**:
1. Очистка и объединение данных из разных источников
2. Feature engineering
3. Обучение модели машинного обучения
4. Визуализация результатов
5. Оптимизация производительности

---

## **2. Архитектура решения**
```mermaid
graph TD
    A[Raw Data] --> B[ETL Pipeline]
    B --> C[Feature Store]
    C --> D[ML Training]
    D --> E[Model Serving]
    E --> F[Dashboard]
```

---

## **3. Реализация проекта**

### **3.1. Инициализация Spark-сессии с оптимизациями**
```python
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CustomerChurnPrediction") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .getOrCreate()
```

### **3.2. ETL-пайплайн**
```python
# 1. Загрузка данных
customers = spark.read.csv("s3://data-lake/customers/*.csv", header=True)
interactions = spark.read.json("s3://data-lake/interactions/*.json")
regions = spark.read.parquet("s3://data-lake/regions/")

# 2. Очистка данных
from pyspark.sql.functions import when, col

clean_customers = customers.withColumn(
    "churn",
    when(col("status") == "cancelled", 1).otherwise(0)
).drop("status")

# 3. Обогащение данных
from pyspark.sql.functions import datediff, current_date

enriched = clean_customers.join(regions, "region_id") \
    .withColumn("tenure_days", datediff(current_date(), col("join_date")))
```

### **3.3. Feature Engineering**
```python
from pyspark.ml.feature import (
    VectorAssembler,
    StringIndexer,
    OneHotEncoder
)

# Категориальные признаки
cat_cols = ["plan_type", "device_type"]
indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx") for c in cat_cols]
encoders = [OneHotEncoder(inputCol=f"{c}_idx", outputCol=f"{c}_enc") for c in cat_cols]

# Числовые признаки
num_cols = ["tenure_days", "monthly_charges", "total_charges"]
assembler = VectorAssembler(inputCols=num_cols + [f"{c}_enc" for c in cat_cols], outputCol="features")

# Пайплайн преобразований
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler])
model = pipeline.fit(enriched)
features = model.transform(enriched)
```

### **3.4. Обучение модели**
```python
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Разделение данных
train, test = features.randomSplit([0.8, 0.2], seed=42)

# Инициализация модели
gbt = GBTClassifier(labelCol="churn", featuresCol="features")

# Подбор гиперпараметров
param_grid = (ParamGridBuilder()
    .addGrid(gbt.maxDepth, [3, 5])
    .addGrid(gbt.maxIter, [50, 100])
    .build())

# Кросс-валидация
evaluator = BinaryClassificationEvaluator(labelCol="churn")
cv = CrossValidator(estimator=gbt, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Обучение
cv_model = cv.fit(train)

# Оценка
predictions = cv_model.transform(test)
auc = evaluator.evaluate(predictions)
print(f"AUC-ROC: {auc:.3f}")
```

### **3.5. Визуализация результатов**
```python
import matplotlib.pyplot as plt
import pandas as pd

# Конвертация в Pandas для визуализации
pd_results = predictions.select("churn", "prediction").toPandas()

# ROC Curve
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(pd_results["churn"], pd_results["prediction"])
plt.plot(fpr, tpr)
plt.title(f"ROC Curve (AUC = {auc:.3f})")
plt.savefig("roc_curve.png")
```

### **3.6. Деплой модели**
```python
# Сохранение пайплайна и модели
cv_model.write().overwrite().save("s3://model-bucket/churn_model")

# Загрузка для предсказаний
from pyspark.ml import PipelineModel
loaded_model = PipelineModel.load("s3://model-bucket/churn_model")

# Пример использования
new_data = spark.read.parquet("s3://data-lake/new_customers/")
predictions = loaded_model.transform(new_data)
```

---

## **4. Оптимизация производительности**
### **4.1. Настройка партиционирования**
```python
features.repartition(100, "region_id").write.parquet(
    "s3://feature-store/churn-features",
    partitionBy="region_id",
    mode="overwrite"
)
```

### **4.2. Кэширование часто используемых данных**
```python
spark.sql("CACHE TABLE customers_features")
```

### **4.3. Мониторинг через Spark UI**
```python
# Доступно по адресу:
# http://<driver-node>:4040
```

---

## **5. Задание для самостоятельной доработки**
1. **Добавьте обработку потоковых данных** (новые взаимодействия клиентов)
2. **Реализуйте A/B тестирование** моделей
3. **Настройте алертинг** при падении качества модели
4. **Оптимизируйте стоимость** выполнения пайплайна

**Пример решения для потоковой обработки**:
```python
streaming_data = spark.readStream \
    .format("kafka") \
    .option("subscribe", "customer-interactions") \
    .load()

def process_stream(batch_df, batch_id):
    # Ваша логика обработки
    pass

query = streaming_data.writeStream \
    .foreachBatch(process_stream) \
    .start()
```

---

## **6. Чеклист завершения проекта**
- [ ] ETL-пайплайн для сырых данных
- [ ] Feature Store с актуальными признаками
- [ ] Обученная и протестированная модель
- [ ] Визуализация ключевых метрик
- [ ] Документация к решению
- [ ] Настройки мониторинга

---

## **7. Дальнейшее развитие проекта**
1. **Интеграция с MLflow** для трекинга экспериментов
2. **Реализация CI/CD** для пайплайна
3. **Переход на Delta Lake** для надежного хранения
4. **Автоматическое переобучение** модели

**Поздравляю!** Вы завершили интенсивный курс по PySpark и теперь готовы к реальным задачам в области больших данных.